# 2진분류의 확장 다중 분류 문제 해결
* 이진 분류(Binary Classification):
    * 정의: 종속 변수가 두 개의 클래스로 구성된 지도 학습 문제로, 두 클래스 중 하나에 속하는지 예측하는 문제입니다.
    * 목적: 주어진 입력 데이터를 기반으로 두 개의 클래스 중 하나로 분류하여 판단하고 예측하는 것을 목표로 합니다.    
  
* 다중 분류(Multiclass Classification):
    * 정의: 종속 변수가 두 개 이상의 범주로 구성된 지도 학습 문제로, 여러 클래스 중 하나에 속하는지 예측하는 문제입니다.
    * 목적: 주어진 입력 데이터를 기반으로 여러 클래스 중 하나로 분류하여 판단하고 예측하는 것을 목표로 합니다.

## 붓꽃 문제

In [30]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# 데이터 로드
iris = load_iris()
X, y = iris.data, iris.target

In [31]:
iris.data[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [32]:
iris.target[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [33]:
set(iris.target) # label의 유일값을 확인해 보니 0,1,2 다중 클래스를 포함하고 있다.

{0, 1, 2}

In [34]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Feature 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
# 4줄 mean, std에 관한
X_test_scaled = scaler.transform(X_test)

# 모델 초기화
knn_classifier = KNeighborsClassifier(n_neighbors=3)

# 모델 훈련
knn_classifier.fit(X_train_scaled, y_train)

# 테스트 데이터로 예측
y_pred = knn_classifier.predict(X_test_scaled)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred) # 인자1: 실제 라벨값, 인자2: 예측 라벨값
print("Accuracy:", accuracy)

Accuracy: 0.9333333333333333


# 실습

## 펭귄 종류 예측 모델

* 아래 데이터 셋을 활용하여 펭귄의 종류를 예측해 보세요.

In [35]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
penguins = sns.load_dataset('penguins')

In [36]:
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


## 전처리 mission

* 결측치는 drop
* Feature 선정
    * 문자 데이터 중에 ordinal 데이터는 포함, nominal 데이터는 불포함
* Label(Target) 정보에 대한 전처리
    * species: 라벨

In [37]:
penguins.dropna(inplace=True)

In [38]:
penguins.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,333.000000,333.000000,333.000000,333.000000
mean,43.992793,17.164865,200.966967,4207.057057
std,5.468668,1.969235,14.015765,805.215802
min,32.100000,13.100000,172.000000,2700.000000
25%,39.500000,15.600000,190.000000,3550.000000
50%,44.500000,17.300000,197.000000,4050.000000
75%,48.600000,18.700000,213.000000,4775.000000
max,59.600000,21.500000,231.000000,6300.000000


In [39]:
penguins['species'].value_counts()

species
Adelie       146
Gentoo       119
Chinstrap     68
Name: count, dtype: int64

In [40]:
penguins.replace('Adelie', 0, inplace=True)

In [41]:
penguins.replace('Gentoo', 1, inplace=True)

In [42]:
penguins.replace('Chinstrap', 2, inplace=True)

In [43]:
feature = penguins[penguins.columns.difference(['species','island','sex'])]
label = penguins['species']

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [45]:
train_input, test_input, train_target, test_target = train_test_split(
    feature, label,stratify=label, random_state=265)

In [46]:
print(train_target.shape, test_target.shape)

(249,) (84,)


In [47]:
print(len(train_input), len(test_input))

249 84


In [48]:
print(test_target)

45     0
213    2
95     0
337    1
88     0
      ..
187    2
117    0
148    0
48     0
255    1
Name: species, Length: 84, dtype: int64


In [49]:
mean = np.mean(train_input, axis=0)
std = np.std(train_input, axis=0)

In [56]:
print(mean, std)

bill_depth_mm          17.165462
bill_length_mm         43.941767
body_mass_g          4170.582329
flipper_length_mm     200.690763
dtype: float64 bill_depth_mm          2.024591
bill_length_mm         5.452985
body_mass_g          781.407675
flipper_length_mm     13.750676
dtype: float64


In [57]:
train_scaled=(train_input-mean)/std

In [58]:
test_scaled = ((test_input) - mean) / std

In [59]:
kn = KNeighborsClassifier()

In [60]:
kn.fit(train_input, train_target)

KNeighborsClassifier()

In [61]:
kn.score(train_input, train_target)

0.8473895582329317

In [62]:
kn.fit(train_scaled, train_target)

KNeighborsClassifier()

In [63]:
kn.score(test_scaled, test_target)

0.9761904761904762